## 7.3 Joint Hypothesis Testing Using the F-Statistic

- The estimated model is

\begin{equation}
\widehat{TestScore} = \underset{(15.21)}{649.58} -\underset{(0.48)}{0.29} \times size - \underset{(0.04)}{0.66} \times english + \underset{(1.41)}{3.87} \times expenditure
\end{equation}

In [3]:
using FixedEffects #FixedEffectModels requires FixedEffects as a dependency
using FixedEffectModels #we use FixedEffectModels to create regression models
using CSV #we use the CSV package to load the data
using DataFrames #we use the DataFrames package as the data is stored as an object of type "DataFrame"
using Plots #we use the Plots package for generating plots

data = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/CASchools.csv") #load the data into the workspace and store it in the variable "data"

data.student_teacher_ratio = data.students ./ data.teachers #add a new column "student_teacher_ratio" to the data, ./ is used to broadcast the division operator between arrays (or in this case, columns)
data.score = (data.read .+ data.math) ./ 2 #add a new column "score" to the data
data.expenditure = data.expenditure ./ 1000 #scale expenditure to thousands of dollars

reg_mod = reg( #initialise a FixedEffectModel and define it as reg_mod
                        data, #pass the DataFrame 'data' as the dataset to be used in reg_mod
                        @model(score ~ student_teacher_ratio + english + expenditure) #pass the regression formula consisting of the dependent variable 'score' and the exogenous variable 'student_teacher_ratio'
)

                                  Linear Model                                  
Number of obs:                      420   Degrees of freedom:                   4
R2:                               0.437   R2 Adjusted:                      0.433
F Statistic:                   0.597073   p-value:                          1.000
                        Estimate Std.Error   t value Pr(>|t|) Lower 95% Upper 95%
student_teacher_ratio  -0.286399  0.480523 -0.596015    0.551  -1.23096  0.658157
english                -0.656023 0.0391059  -16.7756    0.000 -0.732892 -0.579153
expenditure               3.8679   1.41212   2.73907    0.006   1.09212   6.64369
(Intercept)              649.578   15.2057   42.7193    0.000   619.688   679.468


---------------------------------------------------------------------------------


- Now, can we reject the hypothesis that the coefficient on size (the student teacher ratio) and the coefficient on expenditure are zero? 
- To answer this, we have to resort to joint hypothesis tests. 
- A joint hypothesis imposes restrictions on multiple regression coefficients. This is different from conducting individual t-tests where a restriction is imposed on a single coefficient. 
- The homoskedasticity-only F-Statistic is given by

\begin{equation}
F = \frac{(SSR_{\text{restricted}} - SSR_{\text{unrestricted}})/q}{SSR_{\text{unrestricted}} / (n-k-1)}
\end{equation}

- $SSR_{restricted}$ is the sum of squared residuals from the restricted regression, i.e., the regression where we impose the restriction and $SSR_{unrestricted}$ is the sum of squared residuals from the full model, where q is the number of restrictions under the null and k is the number of regressors in the unrestricted regression.
- However, the application of this formula is limited as it is only valid for homoskedastic data.
- Below is a function that calculates the robust F-Statistic that is valid for both homoskedastic and heteroskedastic data.

In [ ]:
function robust_f_test(reg_model, restricted_variable_array, reg_model_name)

    num_restrictions = length(restricted_variable_array)
    num_coef = length(reg_model.coef)

    restriction_matrix = zeros(num_restrictions, num_coef)

    for i in 1:num_restrictions
        restricted_variable_index = findall(x -> x == restricted_variable_array[i], reg_model.coefnames)[1]

        restriction_matrix[i, restricted_variable_index] = 1

    end

    restricted_variable_coef_matrix = restriction_matrix * reg_model.coef


    f_statistic = transpose(restricted_variable_coef_matrix) * inv(restriction_matrix * reg_model.vcov * transpose(restriction_matrix)) * restricted_variable_coef_matrix / num_restrictions

    f_dist = FDist(num_restrictions, Inf64)

    f_statistic_p_value = 1 - cdf(f_dist, f_statistic)

    println("Robust F-Test on the model " * string(reg_model_name) * " with restrictions " * string(restricted_variable_array))
    println("F-Statistic: " * string(f_statistic))
    println("F-Statistic P-Value: " * string(f_statistic_p_value))
    println()

end

- Using this function, we can test whether or not the coefficient on size and the coefficient on expenditure are zero.

In [ ]:
using FixedEffects #FixedEffectModels requires FixedEffects as a dependency
using FixedEffectModels #we use FixedEffectModels to create regression models
using CSV #we use the CSV package to load the data
using DataFrames #we use the DataFrames package as the data is stored as an object of type "DataFrame"
using Plots #we use the Plots package for generating plots
using Distributions

data = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/CASchools.csv") #load the data into the workspace and store it in the variable "data"

data.student_teacher_ratio = data.students ./ data.teachers #add a new column "student_teacher_ratio" to the data, ./ is used to broadcast the division operator between arrays (or in this case, columns)
data.score = (data.read .+ data.math) ./ 2 #add a new column "score" to the data
data.expenditure = data.expenditure ./ 1000 #scale expenditure to thousands of dollars

reg_mod = reg( #initialise a FixedEffectModel and define it as reg_mod
                        data, #pass the DataFrame 'data' as the dataset to be used in reg_mod
                        @model(score ~ student_teacher_ratio + english + expenditure) #pass the regression formula consisting of the dependent variable 'score' and the exogenous variable 'student_teacher_ratio'
)

function robust_f_test(reg_model, restricted_variable_array, reg_model_name)

    num_restrictions = length(restricted_variable_array)
    num_coef = length(reg_model.coef)

    restriction_matrix = zeros(num_restrictions, num_coef)

    for i in 1:num_restrictions
        restricted_variable_index = findall(x -> x == restricted_variable_array[i], reg_model.coefnames)[1]

        restriction_matrix[i, restricted_variable_index] = 1

    end

    restricted_variable_coef_matrix = restriction_matrix * reg_model.coef


    f_statistic = transpose(restricted_variable_coef_matrix) * inv(restriction_matrix * reg_model.vcov * transpose(restriction_matrix)) * restricted_variable_coef_matrix / num_restrictions

    f_dist = FDist(num_restrictions, Inf64)

    f_statistic_p_value = 1 - cdf(f_dist, f_statistic)

    println("Robust F-Test on the model " * string(reg_model_name) * " with restrictions " * string(restricted_variable_array))
    println("F-Statistic: " * string(f_statistic))
    println("F-Statistic P-Value: " * string(f_statistic_p_value))
    println()

end

robust_f_test(reg_mod, ["student_teacher_ratio", "expenditure"], "Model 1")

- The output reveals that the F-statistic for this joint hypothesis test is about $8.01$ and the corresponding p-value is $0.0004$. 
- Thus, we can reject the null hypothesis that both coefficients are zero at any level of significance commonly used in practice.